# Chapter 4
# Response surface methodology: Tuning continuous parameters 

In [ ]:
import numpy as np
import scipy
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
from e4e import E4E

e4e = E4E(chapter=4)

## 4.1	Tune a single continuous parameter

### 4.1.1	Design the experiment

#### Simulate a proprietary trading strategy

In [ ]:
# Listing 4.1 Simulate a markout
def markout_profit(threshold):
    cost = 1
    pps = 1
    signal = np.random.normal()
    eps = 2*np.random.normal()
    if (signal > threshold
        or signal < -threshold):
        profit = pps*np.abs(signal) - cost + eps
    else:
        profit = 0
    return profit

In [ ]:
np.random.seed(17)
profit = np.array([markout_profit(threshold=1) for _ in range(10000)])
i = np.where(profit!=0)[0]
print (len(i), (len(profit)-len(i))/len(profit))
print(profit.mean(), profit.std())
print(profit[i].mean(), profit[i].std())
plt.hist(profit[i], 25, color=e4e.color_1);
plt.xlabel('markout profit')
plt.ylabel('count')
e4e.save_fig(3)

#### CHOOSE THE PARAMETER VALUES

#### CONTINUOUS PARAMETERS IN RSM

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])
ax3.set_xticks([])
ax3.set_yticks([])
ax4.set_xticks([])
ax4.set_yticks([])

x = [1,2]
y = [1,2]
ax1.plot(x,y, 'o-', color=e4e.color_1);
ax1.axis([.5, 2.5, .5, 2.5])
ax1.set_xticks([1,2])
ax1.set_xticklabels('')
ax1.text(.6,2.2,'(a)')

x = [1,2]
y = [2,1]
ax2.plot(x,y, 'o-', color=e4e.color_1);
ax2.axis([.5, 2.5, .5, 2.5])
ax2.set_xticks([1,2])
ax2.set_xticklabels('')
ax2.text(.6,2.2,'(b)')

x = np.arange(1,2.001,.01)
x0 = 1.75
y = 2 - 2*(x-x0)**2
ax3.plot(x,y, '-',color=e4e.color_1);
ax3.plot([x[0], x[50], x[-1]], [y[0], y[50], y[-1]], 'o', color=e4e.color_1)
i=np.where(np.abs(x-x0)<.001)[0]
ax3.plot([x[i]], [y[i]], 'o', color=e4e.color_1, fillstyle='none')
ax3.axis([.5, 2.5, .5, 2.5])
ax3.set_xticks([1,1.5,2])
ax3.set_xticklabels('')
ax3.text(.6,2.2,'(c)')

x = np.arange(1,2.001,.01)
x0 = 1.25
y = 2 - 2*(x-x0)**2
ax4.plot(x,y, '-',color=e4e.color_1);
ax4.plot([x[0], x[50], x[-1]], [y[0], y[50], y[-1]], 'o', color=e4e.color_1)
i=np.where(np.abs(x-x0)<.001)[0]
ax4.plot([x[i]], [y[i]], 'o', color=e4e.color_1, fillstyle='none')
ax4.axis([.5, 2.5, .5, 2.5])
ax4.set_xticks([1,1.5,2])
ax4.set_xticklabels('')
ax4.text(.6,2.2,'(d)')

fig.text(.5, 0, 'threshold', ha='center')
fig.text(0, 0.5, 'profit', va='center', rotation='vertical')

e4e.save_fig(6)

#### DETERMINE THE NUMBER OF INDIVIDUAL MEASUREMENTS

In [ ]:
profit.mean()

In [ ]:
sd_delta = 1.2
print ("sd_delta =", sd_delta, profit.std())
prac_sig = .03
print ("prac_sig = ", prac_sig, .2 * profit.mean())
num_ind = (2.48 * sd_delta/prac_sig)**2
print ("num_ind =", num_ind)
num_ind = (3.08 * sd_delta/prac_sig)**2
print ("num_ind =", num_ind)

#### 4.1.2 Run and analyze the experiment

In [ ]:
# Listing 4.2 Simulate the experiment
def run_experiment(num_ind, thresholds):
    individual_measurements = {
     threshold: [] for threshold in thresholds
    }
    done = set()
    while True:
        threshold = np.random.choice(thresholds)
        profit = markout_profit(threshold)
        individual_measurements[threshold].append(profit)
        if (len(individual_measurements[threshold])
            >= num_ind):
            done.add(threshold)
        if len(done)==len(thresholds):
            break
    
    aggregate_measurements = []
    standard_errors = []
    for threshold in thresholds:
        ims = np.array(individual_measurements[threshold])
        aggregate_measurements.append( ims.mean() )
        standard_errors.append( ims.std()/np.sqrt(len(ims)) )
        
    return aggregate_measurements, standard_errors

In [ ]:
np.random.seed(17)
thresholds = np.array([0.5, 1.0, 1.5])
aggregate_measurements, standard_errors = run_experiment(15000, thresholds)

In [ ]:
plt.errorbar(thresholds, aggregate_measurements,
             yerr=standard_errors,
             fmt='o', color=e4e.color_1, capsize=10);
plt.xlabel('threshold')
plt.ylabel('markout_profit')

e4e.save_fig(8)

In [ ]:
# Listing 4.3 Fit a one-parameter model using linear regression
def linear_regression(thresholds, aggregate_measurements):
    x = thresholds
    y = aggregate_measurements
    X = np.array([np.ones(len(y)), x, x**2]).T
    beta = np.linalg.inv(X.T @ X) @ (X.T @ y)
    return beta

In [ ]:
beta = linear_regression(thresholds, aggregate_measurements)
print(beta)

In [ ]:
# Listing 4.4 Interpolate between measurements
def interpolate(thresholds, beta):
    xhat = np.arange(thresholds.min(), thresholds.max()+1e-6, .01)
    XHat = np.array([np.ones(len(xhat)), xhat, xhat**2]).T
    yhat = XHat @ beta
    return xhat, yhat

In [ ]:
plt.errorbar(thresholds, aggregate_measurements,
             yerr=standard_errors,
             fmt='o', color=e4e.color_1, capsize=10);
xhat, yhat = interpolate(thresholds, beta)
plt.plot(xhat, yhat, '--', color=e4e.color_2)
plt.xlabel('threshold')
plt.ylabel('markout profit')

e4e.save_fig(10)

In [ ]:
beta = linear_regression(thresholds, aggregate_measurements)
print(beta)

In [ ]:
# Listing 4.5 Optimize the interpolation function
def optimize(thresholds, beta):
    xhat, yhat = interpolate(thresholds, beta)
    i = np.where(yhat==yhat.max())[0][0]
    return xhat[i], yhat[i]

In [ ]:
threshold_opt, estimated_max_profit = optimize(thresholds, beta)
print (threshold_opt, estimated_max_profit)

In [ ]:
plt.errorbar(thresholds, aggregate_measurements,
             yerr=standard_errors,
             fmt='o', color=e4e.color_1, capsize=10);
xhat, yhat = interpolate(thresholds, beta)
plt.plot(xhat, yhat, '--', color=e4e.color_2)
plt.plot(threshold_opt, estimated_max_profit, 'X',  color=e4e.color_2)
plt.xlabel('threshold')
plt.ylabel('profit')

e4e.save_fig(11)

### 4.1.3	Validate the optimal parameter value

#### A SIMPLE VALIDATION MEASUREMENT

In [ ]:
estimated_max_profit, 

In [ ]:
np.random.seed(17)
aggregate_measurement, standard_error = run_experiment(15000, [threshold_opt])
print (aggregate_measurement[0]-2*standard_error[0], aggregate_measurement[0]+2*standard_error[0])

#### A MORE ROBUST VALIDATION MEASUREMENT

## 4.2	Tune two or more continuous parameters

In [ ]:
# Listing 4.6 Markout profit as a function of threshold and order size
def markout_profit_2D(threshold, order_size):
    cost = 1
    pps = 1
    asc = 0.001*np.exp(2*order_size)
    signal = np.random.normal()
    eps = 2*np.random.normal()
    if (signal > threshold
        or signal < -threshold):
        profit = order_size*(pps*np.abs(signal) - cost + eps) - asc
    else:
        profit = 0
    return profit

## 4.2.1	Design the experiment

In [ ]:
thresholds = [0.5, 1.0, 1.5]
order_sizes = [1, 1.5, 2]
axes = [.45, 1.55, .9, 2.1]
aspect = (thresholds[-1]-thresholds[0]) / (order_sizes[-1]-order_sizes[0])

fig, ((ax1, ax2, ax3)) = plt.subplots(1, 3)
ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])
ax3.set_xticks([])
ax3.set_yticks([])

ax1.plot(thresholds, [1.5, 1.5, 1.5], 'o', color=e4e.color_1);
ax1.axis(axes)
ax1.set_aspect(aspect)
ax1.set_xticks(thresholds)
ax1.text(1.0, .6, 'threshold', ha='center')
ax1.text(0.5, 1.8, '(a)')

ax2.plot([1.0, 1.0, 1.0], order_sizes, 'o', color=e4e.color_1);
ax2.axis(axes)
ax2.set_aspect(aspect)
ax2.set_yticks(order_sizes)
ax2.set_xticks([-10])
ax2.text(1.0, .6, 'threshold', ha='center')
ax2.text(0.5, 1.8, '(b)')

ax3.plot(thresholds, [1.5, 1.5, 1.5], 'o', color=e4e.color_1);
ax3.plot([1.0, 1.0, 1.0], order_sizes, 'o', color=e4e.color_1);
ax3.axis(axes)
ax3.set_aspect(aspect)
ax3.set_xticks(thresholds)
ax3.set_yticks(order_sizes)
ax3.text(1.0, .6, 'threshold', ha='center')
ax3.text(0.5, 1.8, '(c)')

fig.text(0, 0.5, 'order_size', va='center', rotation='vertical')

e4e.save_fig(14)

In [ ]:
# Listing 4.7 Face-centered central composite design
def design_ccd(thresholds, order_sizes):
    parameters = [
        (threshold, order_size)
        for threshold in thresholds
        for order_size in order_sizes
    ]
    return parameters

In [ ]:
parameters = design_ccd(thresholds=[0.5, 1.0, 1.5], order_sizes=[1, 1.5, 2])
print (parameters)

In [ ]:
pp = np.array(parameters)
thresholds = pp[:,0]
order_sizes = pp[:,1]
plt.plot(thresholds, order_sizes, 'o', color=e4e.color_1)
plt.xlabel('threshold')
plt.ylabel('order size')
e4e.save_fig(15)

### 4.2.2	Run, analyze, and validate the experiment

In [ ]:
np.random.seed(17)
profit = np.array([markout_profit_2D(threshold=1, order_size=1) for _ in range(10000)])
i = np.where(profit!=0)[0]
print (len(i), (len(profit)-len(i))/len(profit))
print(profit.mean(), profit.std())

#### RUN THE EXPERIMENT

In [ ]:
# Listing 4.8 Run a 2D experiment
def run_experiment_2D(num_ind, parameters):
    individual_measurements = {
      parameter: [] for parameter in parameters
    }
    done = set()
    while True:
        parameter = parameters[np.random.choice(len(parameters))]
        threshold, order_size = parameter
        profit = markout_profit_2D(threshold, order_size)
        individual_measurements[parameter].append(profit)
        if (len(individual_measurements[parameter])
            >= num_ind):
            done.add(parameter)
        if len(done) == len(individual_measurements):
            break
    
    aggregate_measurements = []
    standard_errors = []
    for parameter in parameters:
        ims = np.array(individual_measurements[parameter])
        aggregate_measurements.append( ims.mean() )
        standard_errors.append( ims.std()/np.sqrt(len(ims)) )
        
    return aggregate_measurements, standard_errors

In [ ]:
np.random.seed(17)
parameters = design_ccd(thresholds=[0.5, 1.0, 1.5], order_sizes=[1, 1.5, 2])
aggregate_measurements, standard_errors = run_experiment_2D(15000, parameters)

In [ ]:
def _plot_aggregate_measurements(parameters, aggregate_measurements, standard_errors):
    n = np.arange(len(parameters))
    plt.errorbar(n, aggregate_measurements,
                 yerr=standard_errors,
                 fmt='o', color=e4e.color_1, capsize=10);
    plt.ylabel('markout profit', fontsize=e4e.font_size_2d)
    plt.yticks(fontsize=e4e.font_size_2d)
    plt.xticks(
        ticks=list(range(len(parameters))),
        labels=[f"th={p[0]:.1f}\nos={p[1]:.1f}" for p in parameters],
        fontsize=e4e.font_size_2d
    )


In [ ]:
_plot_aggregate_measurements(parameters, aggregate_measurements, standard_errors)
e4e.save_fig(17)

#### ANALYZE THE EXPERIMENT

In [ ]:
# Listing 4.9 Linear regression for two parameters
def linear_regression_2D(parameters, aggregate_measurements):
    parameters = np.array(parameters)
    x0 = parameters[:,0]
    x1 = parameters[:,1]
    y = np.array(aggregate_measurements)
    X = np.array([np.ones(len(y)), x0, x1, x0**2, x1**2, x0*x1]).T
    beta = np.linalg.inv(X.T @ X) @ (X.T @ y)
    return beta

In [ ]:
beta = linear_regression_2D(parameters, aggregate_measurements)
print (beta)

In [ ]:
# Listing 4.10 Interpolation for two parameters
def interpolate_2D(parameters, beta):
    parameters = np.array(parameters)
    x0_values = np.arange(parameters[:,0].min(), parameters[:,0].max()+1e-6, .01)
    x1_values = np.arange(parameters[:,1].min(), parameters[:,1].max()+1e-6, .01)
    x0hat_2d, x1hat_2d = np.meshgrid(x0_values, x1_values)
    x0hat = x0hat_2d.flatten()
    x1hat = x1hat_2d.flatten()
    XHat = np.array([np.ones(len(x0hat)), x0hat, x1hat, x0hat**2, x1hat**2, x0hat*x1hat]).T
    yhat = XHat @ beta
    yhat_2d = np.reshape(yhat, (len(x1_values), len(x0_values)))
    return x0hat_2d, x1hat_2d, yhat_2d

In [ ]:
parameters

In [ ]:
def _plot_interpolation(parameters, aggregate_measurements, beta, parameter_opt=None):
    parameters = np.array(parameters)
    thresholds = parameters[:,0].copy()
    order_sizes = parameters[:,1].copy()
    # Hack to expand plot just enough to show full dots
    g = .025
    parameters[0,0] = parameters[0,0]-g
    parameters[0,1] = parameters[0,1]-g
    parameters[-1,0] = parameters[-1,0]+g
    parameters[-1,1] = parameters[-1,1]+g
    import time
    t0 = time.time()
    x0hat, x1hat, yhat = interpolate_2D(parameters, beta)
    tf = time.time()
    print (tf-t0)
    fig = plt.figure()
    plt.contourf(x0hat, x1hat, yhat, alpha=.5, cmap="Greys")
    plt.plot(thresholds, order_sizes, 'o', color=e4e.color_1)
    plt.colorbar()
    plt.title('markout_profit')
    
    if parameter_opt is not None:
        threshold_opt = parameter_opt[0]
        order_size_opt = parameter_opt[1]
        plt.plot(threshold_opt, order_size_opt, 'X', color=e4e.color_1, markersize=10)
    
    plt.xlabel('threshold')
    plt.ylabel('order size')

In [ ]:
_plot_interpolation(parameters, aggregate_measurements, beta)
e4e.save_fig(18)

In [ ]:
# Listing 4.11 Optimize the 2D surrogate function
def optimize_2D(parameters, beta):
    x0hat, x1hat, yhat = interpolate_2D(parameters, beta)
    i = np.where(yhat==yhat.max())
    return x0hat[i][0], x1hat[i][0], yhat[i][0]

In [ ]:
beta = linear_regression_2D(parameters, aggregate_measurements)
threshold_opt, order_size_opt, estimated_max_profit = optimize_2D(parameters, beta)
print (threshold_opt, order_size_opt, estimated_max_profit)

In [ ]:
parameters = design_ccd(thresholds=[.5,  1,  1.5], order_sizes=[2.5, 3, 3.5])
np.random.seed(17)
aggregate_measurements, standard_errors = run_experiment_2D(15000, parameters)
aggregate_measurements_prev, standard_errors_prev = aggregate_measurements, standard_errors

_plot_aggregate_measurements(parameters, aggregate_measurements, standard_errors)
axis_prev = plt.axis()
e4e.save_fig(19)

In [ ]:
parameters = design_ccd(thresholds=[.75,  1, 1.25], order_sizes=[2.75, 3, 3.25])
np.random.seed(17)
aggregate_measurements, standard_errors = run_experiment_2D(15000, parameters)

_plot_aggregate_measurements(parameters, aggregate_measurements, standard_errors)
c = plt.axis()
plt.axis([c[0], c[1], axis_prev[2], axis_prev[3]])
e4e.save_fig(20)

In [ ]:
beta = linear_regression_2D(parameters, aggregate_measurements)
threshold_opt, order_size_opt, estimated_max_profit = optimize_2D(parameters, beta)
print (threshold_opt, order_size_opt, estimated_max_profit)

ax = _plot_interpolation(parameters, aggregate_measurements, beta, (threshold_opt, order_size_opt))
e4e.save_fig(21)

#### VALIDATE THE INTERPOLATION ESTIMATE

In [ ]:
np.random.seed(17)
aggregate_measurement, standard_error = run_experiment_2D(
    num_ind=15000,
    parameters=[(threshold_opt, order_size_opt)]
)

In [ ]:
print (aggregate_measurement, standard_error)
print(aggregate_measurement[0] - 2*standard_error[0], aggregate_measurement[0] + 2*standard_error[0])